# Install Required Libraries

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Model Configuration for LLaMA-2 Fine-tuning with QLoRA
The code defines configuration parameters for fine-tuning LLaMA-2 using QLoRA, including model specifications, quantization settings, training arguments, and sequence handling parameters.

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-hf"

# The instruction dataset to use
dataset_name = "jamescalam/ai-arxiv2-chunks"

# Fine-tuned model name
new_model = "/content/Llama-2-7b-ai_arxiv-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 2

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 1024

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# Dataset Splitting and Processing
Function to split arXiv dataset into train/test sets by paper DOIs, preserving all text chunks from each paper. Loads dataset, performs split, and saves resulting JSONL files.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

def create_paper_splits(df, test_size=0.2, random_state=42):
    """
    Split dataset by papers (doi) preserving all chunks
    """
    # Split papers
    unique_papers = df['doi'].unique()
    train_papers, test_papers = train_test_split(
        unique_papers,
        test_size=test_size,
        random_state=random_state
    )

    # Create train and test sets keeping all chunks
    train_data = df[df['doi'].isin(train_papers)].copy()
    test_data = df[df['doi'].isin(test_papers)].copy()

    # Print statistics
    print(f"Train set - Papers: {len(train_papers)}, Chunks: {len(train_data)}")
    print(f"Test set - Papers: {len(test_papers)}, Chunks: {len(test_data)}")

    return train_data, test_data

# Load and split dataset
df = pd.read_json("hf://datasets/jamescalam/ai-arxiv2-chunks/train.jsonl", lines=True)
train_data, test_data = create_paper_splits(df)

# Save splits
train_data.to_json("train_data.jsonl", orient='records', lines=True)
test_data.to_json("test_data.jsonl", orient='records', lines=True)

In [ ]:
import pandas as pd
from datasets import Dataset

def prepare_dataset_max(json_path, max_samples):
    df = pd.read_json(json_path, lines=True)
    df = df.head(max_samples)
    return Dataset.from_pandas(df)

In [ ]:
dataset = prepare_dataset_max(json_path="/content/train_data.jsonl", max_samples = 1000)

In [ ]:
dataset[0]

{'doi': 2401.04088,
 'chunk-id': 0,
 'chunk': '4 2 0 2\nn a J 8 ] G L . s c [\n1 v 8 8 0 4 0 . 1 0 4 2 : v i X r a\n# Mixtral of Experts\nAlbert Q. Jiang, Alexandre Sablayrolles, Antoine Roux, Arthur Mensch, Blanche Savary, Chris Bamford, Devendra Singh Chaplot, Diego de las Casas, Emma Bou Hanna, Florian Bressand, Gianna Lengyel, Guillaume Bour, Guillaume Lample, LÃ©lio Renard Lavaud, Lucile Saulnier, Marie-Anne Lachaux, Pierre Stock, Sandeep Subramanian, Sophia Yang, Szymon Antoniak, Teven Le Scao, ThÃ©ophile Gervet, Thibaut Lavril, Thomas Wang, TimothÃ©e Lacroix, William El Sayed',
 'id': '2401.04088#0',
 'title': 'Mixtral of Experts',
 'summary': 'We introduce Mixtral 8x7B, a Sparse Mixture of Experts (SMoE) language model.\nMixtral has the same architecture as Mistral 7B, with the difference that each\nlayer is composed of 8 feedforward blocks (i.e. experts). For every token, at\neach layer, a router network selects two experts to process the current state\nand combine their outpu

## Formatting function for respective dataset

In [ ]:
def formatting_func(example):
    formatted_text = f"""### DOI: {example['doi']}
### Chunk ID: {example['chunk-id']}
### ID: {example['id']}
### Title: {example['title']}
### Summary: {example['summary']}
### Source: {example['source']}
### Authors: {example['authors']}
### Categories: {example['categories']}
### Comment: {example['comment']}
### Journal Reference: {example['journal_ref']}
### Primary Category: {example['primary_category']}
### Published: {example['published']}
### Updated: {example['updated']}
### References: {example['references']}
### Content: {example['chunk']}"""
    return {"text": formatted_text}

In [ ]:
formatting_func(dataset[0])

{'text': "### DOI: 2401.04088\n### Chunk ID: 0\n### ID: 2401.04088#0\n### Title: Mixtral of Experts\n### Summary: We introduce Mixtral 8x7B, a Sparse Mixture of Experts (SMoE) language model.\nMixtral has the same architecture as Mistral 7B, with the difference that each\nlayer is composed of 8 feedforward blocks (i.e. experts). For every token, at\neach layer, a router network selects two experts to process the current state\nand combine their outputs. Even though each token only sees two experts, the\nselected experts can be different at each timestep. As a result, each token has\naccess to 47B parameters, but only uses 13B active parameters during inference.\nMixtral was trained with a context size of 32k tokens and it outperforms or\nmatches Llama 2 70B and GPT-3.5 across all evaluated benchmarks. In particular,\nMixtral vastly outperforms Llama 2 70B on mathematics, code generation, and\nmultilingual benchmarks. We also provide a model fine-tuned to follow\ninstructions, Mixtral 8

In [ ]:
tokenized_dataset = dataset.map(formatting_func, remove_columns=dataset.column_names, batched=False)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

# Model and Tokenizer Initialization
Sets up LLaMA model with 4-bit quantization, configures tokenizer with padding settings, and checks GPU compatibility for bfloat16 training acceleration.

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

# Training Configuration and Execution
Configures LoRA parameters, training arguments including optimization settings, initializes SFT trainer with the model, and starts the fine-tuning process.

In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    formatting_func=formatting_func,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,1.626800
50,1.293900
75,1.112400
100,0.929100
125,0.902000


TrainOutput(global_step=125, training_loss=1.1728577575683594, metrics={'train_runtime': 3792.9289, 'train_samples_per_second': 0.264, 'train_steps_per_second': 0.033, 'total_flos': 2.029735884644352e+16, 'train_loss': 1.1728577575683594, 'epoch': 1.0})

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [ ]:
trainer.tokenizer.save_pretrained(new_model)


('Llama-2-7b-ai_arxiv-finetune/tokenizer_config.json',
 'Llama-2-7b-ai_arxiv-finetune/special_tokens_map.json',
 'Llama-2-7b-ai_arxiv-finetune/tokenizer.model',
 'Llama-2-7b-ai_arxiv-finetune/added_tokens.json',
 'Llama-2-7b-ai_arxiv-finetune/tokenizer.json')

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "You are a AI scholar. Now tell me what is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] You are a AI scholar. Now tell me what is a large language model? [/INST]
[INST] You are a AI scholar. Now tell me what is a large language model? [/INST]
[INST] You are a AI scholar. Now tell me what is a large language model?
[INST] You are a AI scholar. Now tell me what is a large language model? [/INST]
[INST] You are a AI scholar. Now tell me what is a large language model? [/INST]
[INST] You are a AI scholar. Now tell me what is a large language model? [/INST]
[INST] You are a AI scholar. Now tell me what is a large language model? [/INST]
[INST] You are a AI scholar. Now tell me what is a large language model? [/INST]
[INST]
